# Data Mining (2CSDE71)
## Name :- Harsh Pansuriya
## Roll No. :- 21BCE176
## Date :- 19th April, 2024
## Prac. No :- 8

### Aim:- 
The DBLP dataset (www.informatik.uni-trier.de/ley/db/) consists of over
one million entries of research papers published in computer science
conferences and journals. Among these entries, there are a good number
of authors that have coauthor relationships. (a) Propose a method and
implement it to efficiently mine a set of coauthor relationships that are
closely correlated (e.g., often coauthoring papers together).


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statistics
from sklearn.preprocessing import LabelEncoder
import itertools

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth

In [3]:
df = pd.read_csv("/Users/harsh/Documents/DM_prac/Database/dblp_data.csv")
print(df.head(10))
print(df.info())

   id                  author  experience  num_of_papers  \
0   0             'Anau Mesui           4              1   
1   1         'Yinka Oyerinde           4              2   
2   2            A'fza Shafie           4              1   
3   3         A'lishia Bowman           4              1   
4   4               A'na Wang           4              5   
5   5  A'zraa Afhzan Ab Rahim           4              1   
6   6       A. A. Pukhal'skii           4              1   
7   7            A. Al-Ta'ani           4              1   
8   8        A. B. Ugol'nikov           4              1   
9   9          A. D'Agapayeff           4              1   

                  conference                                          subjects  
0                  ['ICT4D']                                         ['Other']  
1      ['CONF-IRM', 'ICT4D']                                         ['Other']  
2                 ['NGMAST']                                         ['Other']  
3              

In [4]:
df = df.head(500)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             500 non-null    int64 
 1   author         500 non-null    object
 2   experience     500 non-null    int64 
 3   num_of_papers  500 non-null    int64 
 4   conference     500 non-null    object
 5   subjects       500 non-null    object
dtypes: int64(3), object(3)
memory usage: 23.6+ KB
None


In [5]:
# Select only the 'author' and 'conference' columns
df_selected_columns = df[['author', 'conference']]

# Print the first few rows of the selected DataFrame
print(df_selected_columns.head())

            author                 conference
0      'Anau Mesui                  ['ICT4D']
1  'Yinka Oyerinde      ['CONF-IRM', 'ICT4D']
2     A'fza Shafie                 ['NGMAST']
3  A'lishia Bowman                   ['GHTC']
4        A'na Wang  ['IIKI', 'CWSN', 'COCOA']


In [6]:
df = df_selected_columns
lst = []
for i in df['conference']:
    s=""
    for j in i:
        if j=='[' or j==']' :
            s=s+' '
        elif j == "'":
            s=s+' '
        else:
            s=s+j
        s = s.replace(" ", "")
    lst.append(s)
print(lst)

['ICT4D', 'CONF-IRM,ICT4D', 'NGMAST', 'GHTC', 'IIKI,CWSN,COCOA', 'CICSYN', 'PERFORMANCE', 'ICNS', 'FCT', 'ICSE', 'PCFD', 'ISSCC', 'SWAP,IWASI', 'MIE', 'IMACS', 'EGITALY', 'DFT', 'DH', 'EUROSIM', 'ISCAS', 'IFIP', 'VLSID,ATS', 'PSCC', 'CIIA', '3DIC', 'IFIP', 'EUCC', 'ADAEUROPE', 'JCDL', 'MAMMO', 'OFC', 'ESSDERC', 'GLOBECOM,ICUMT', 'FRACTALS', 'IFIP', 'IWCMC', 'MEDINFO', 'ICSM', 'IROS,ICML,NIPS,ICRA', 'PPSC', 'SICE', 'IOTSMS', 'IOTSMS', 'ISCC,ICT,ICNS,PROCEDIA', 'VW,ISWSA', 'ICSOFT', 'CISSE', 'IECON', 'IHCI', 'ISSPA,ICSIPA', 'ICCSCE', 'IADIS', 'FNC', 'ICUFN,ICFNDS', 'VTC', 'IECON', 'INNS-WC,ICT-EURASIA,CF,EMS,DAENG,CVC', 'CITA', 'PROCEDIA,CCECE,IEEEIAS,CCNC,ASUNAM,DOCENG,ICPPW,TREC,EMNLP,COLING,CGI,HICSS', 'SOCPAR,SITIS,SKIMA', 'CGO,DESEC', 'WORLDCIST', 'ICICDT', 'APCCAS,EDUCON,MOCAST,ATS', 'APCC,AIMECH', 'ICTC', 'IJCNN', 'VTC', 'IJCNN', 'SP', 'AMIA', 'ICC,CCS,PADS,CEAS', 'CVPR,ICASSP', 'EMBC', 'ICMC,NIME', 'ER', 'MIBAM,MICAD', 'GEM', 'POS', 'CIKM,ITI,KDWEB,BDC,BIGDATASERVICE', 'IIWAS', '

In [7]:
lst_lst = []
for i in lst:
    split_strings = i.strip().split(',')
    lst_lst.append(split_strings)
print(lst_lst)


[['ICT4D'], ['CONF-IRM', 'ICT4D'], ['NGMAST'], ['GHTC'], ['IIKI', 'CWSN', 'COCOA'], ['CICSYN'], ['PERFORMANCE'], ['ICNS'], ['FCT'], ['ICSE'], ['PCFD'], ['ISSCC'], ['SWAP', 'IWASI'], ['MIE'], ['IMACS'], ['EGITALY'], ['DFT'], ['DH'], ['EUROSIM'], ['ISCAS'], ['IFIP'], ['VLSID', 'ATS'], ['PSCC'], ['CIIA'], ['3DIC'], ['IFIP'], ['EUCC'], ['ADAEUROPE'], ['JCDL'], ['MAMMO'], ['OFC'], ['ESSDERC'], ['GLOBECOM', 'ICUMT'], ['FRACTALS'], ['IFIP'], ['IWCMC'], ['MEDINFO'], ['ICSM'], ['IROS', 'ICML', 'NIPS', 'ICRA'], ['PPSC'], ['SICE'], ['IOTSMS'], ['IOTSMS'], ['ISCC', 'ICT', 'ICNS', 'PROCEDIA'], ['VW', 'ISWSA'], ['ICSOFT'], ['CISSE'], ['IECON'], ['IHCI'], ['ISSPA', 'ICSIPA'], ['ICCSCE'], ['IADIS'], ['FNC'], ['ICUFN', 'ICFNDS'], ['VTC'], ['IECON'], ['INNS-WC', 'ICT-EURASIA', 'CF', 'EMS', 'DAENG', 'CVC'], ['CITA'], ['PROCEDIA', 'CCECE', 'IEEEIAS', 'CCNC', 'ASUNAM', 'DOCENG', 'ICPPW', 'TREC', 'EMNLP', 'COLING', 'CGI', 'HICSS'], ['SOCPAR', 'SITIS', 'SKIMA'], ['CGO', 'DESEC'], ['WORLDCIST'], ['ICICDT'], [

In [8]:
unique_conference = []
for i in lst_lst:
    for j in i:
        if j not in unique_conference:
            unique_conference.append(j)

unique_conference.sort()  # Sort the list in-place
print(unique_conference)

['3DIC', '3DIM', 'AAAI', 'AAAIFS', 'AAAISS', 'AAMAS', 'ACIIDS', 'ACIVS', 'ACL', 'ACL-CODESWITCH', 'ACL-INSIGHTS', 'ACL-LDL', 'ACL-LOUHI', 'ACL-SIGHAN', 'ACL-TRAC', 'ACLWAT', 'ACMACE', 'ACMDIS', 'ACMICEC', 'ACMIDC', 'ADAEUROPE', 'ADBIS', 'ADHOCNETS', 'ADHS', 'AE', 'AFIPS', 'AGI', 'AH', 'AHS', 'AI', 'AICCSA', 'AICT', 'AIES', 'AIIA', 'AIME', 'AIMECH', 'AIMSA', 'AINA', 'AISI', 'AIST', 'AISTATS', 'ALDT', 'ALIFE', 'AMCC', 'AMCIS', 'AMFG', 'AMI', 'AMIA', 'AMR', 'ANT', 'APCC', 'APCCAS', 'APPLEPIES', 'ARTSIT', 'ASCC', 'ASMS-SPSC', 'ASPLOS', 'ASSCC', 'ASSETS', 'ASTROINF', 'ASUNAM', 'ATAL', 'ATMN', 'ATMOS', 'ATS', 'AUGHUMAN', 'AVI', 'AVR', 'BCB', 'BDAW', 'BDC', 'BEA', 'BERKELEY', 'BESC', 'BIBE', 'BIBM', 'BIC-TA', 'BICA', 'BIGDATA', 'BIGDATACONF', 'BIGDATASERVICE', 'BIOCAS', 'BIOSTEC', 'BIRTHDAY', 'BLED', 'BPM', 'BTAS', 'BWCCA', 'CAIA', 'CAIP', 'CAISE', 'CAMAD', 'CAMSAP', 'CARS', 'CASCON', 'CASE', 'CATA', 'CCECE', 'CCGRID', 'CCN', 'CCNC', 'CCS', 'CD', 'CDC', 'CDS', 'CEAS', 'CEEC', 'CF', 'CGI', 'CG

In [9]:
print("Unique number of conferences are : ",len(unique_conference))

Unique number of conferences are :  718


In [10]:
dict_conf_id = {}
id_conf = 1
for i in unique_conference:
    dict_conf_id[i]=id_conf
    id_conf += 1
print(dict_conf_id)

{'3DIC': 1, '3DIM': 2, 'AAAI': 3, 'AAAIFS': 4, 'AAAISS': 5, 'AAMAS': 6, 'ACIIDS': 7, 'ACIVS': 8, 'ACL': 9, 'ACL-CODESWITCH': 10, 'ACL-INSIGHTS': 11, 'ACL-LDL': 12, 'ACL-LOUHI': 13, 'ACL-SIGHAN': 14, 'ACL-TRAC': 15, 'ACLWAT': 16, 'ACMACE': 17, 'ACMDIS': 18, 'ACMICEC': 19, 'ACMIDC': 20, 'ADAEUROPE': 21, 'ADBIS': 22, 'ADHOCNETS': 23, 'ADHS': 24, 'AE': 25, 'AFIPS': 26, 'AGI': 27, 'AH': 28, 'AHS': 29, 'AI': 30, 'AICCSA': 31, 'AICT': 32, 'AIES': 33, 'AIIA': 34, 'AIME': 35, 'AIMECH': 36, 'AIMSA': 37, 'AINA': 38, 'AISI': 39, 'AIST': 40, 'AISTATS': 41, 'ALDT': 42, 'ALIFE': 43, 'AMCC': 44, 'AMCIS': 45, 'AMFG': 46, 'AMI': 47, 'AMIA': 48, 'AMR': 49, 'ANT': 50, 'APCC': 51, 'APCCAS': 52, 'APPLEPIES': 53, 'ARTSIT': 54, 'ASCC': 55, 'ASMS-SPSC': 56, 'ASPLOS': 57, 'ASSCC': 58, 'ASSETS': 59, 'ASTROINF': 60, 'ASUNAM': 61, 'ATAL': 62, 'ATMN': 63, 'ATMOS': 64, 'ATS': 65, 'AUGHUMAN': 66, 'AVI': 67, 'AVR': 68, 'BCB': 69, 'BDAW': 70, 'BDC': 71, 'BEA': 72, 'BERKELEY': 73, 'BESC': 74, 'BIBE': 75, 'BIBM': 76, 'BI

In [11]:
transformed_data = []
for author, conference in zip(df['author'],lst_lst):
    add_row = []
    add_row.append(author)
    conf_id_conversion = []
    for i in conference:
        conf_id_conversion.append(dict_conf_id[i])
    add_row.append(conf_id_conversion)
    transformed_data.append(add_row)
print(transformed_data)

[["'Anau Mesui", [366]], ["'Yinka Oyerinde", [150, 366]], ["A'fza Shafie", [533]], ["A'lishia Bowman", [270]], ["A'na Wang", [402, 167, 138]], ["A'zraa Afhzan Ab Rahim", [116]], ["A. A. Pukhal'skii", [555]], ["A. Al-Ta'ani", [347]], ["A. B. Ugol'nikov", [241]], ["A. D'Agapayeff", [357]], ["A. D'Alascio", [553]], ["A. D'Alessandro", [449]], ["A. D'Amico", [658, 467]], ["A. D'Arminio Monforte", [502]], ["A. D'Orta", [407]], ["A. Dell'Accio", [208]], ["A. Dell'Acqua", [181]], ["A. Dell'Aglio", [182]], ["A. Dell'Arte", [231]], ["A. G. O'Neill", [432]], ["A. I. Acil'diev", [393]], ["A. K. B. A'ain", [683, 65]], ["A. L'Abbate", [569]], ["A. M'Hamed", [119]], ["A. M'zah", [1]], ["A. M. Gil'man", [393]], ["A. Noe'", [226]], ["A. O'Brien", [21]], ["A. O'Conner", [473]], ["A. O'Doherty", [490]], ["A. O'Hare", [542]], ["A. O'Neill", [223]], ["A. Okassa-M'foubat", [272, 373]], ["A. R. Vin'a", [256]], ["A. S. Narin'ani", [393]], ["A. d'Ussel", [469]], ["A. van't Hooft", [495]], ["Aad 't Hart", [360

In [12]:
final_transformation = {}
for i in transformed_data:
    for j in i[1]:
        if j not in final_transformation:
            final_transformation[j] = []  # Initialize an empty list if key doesn't exist
        final_transformation[j].append(i[0])

print(final_transformation)


{366: ["'Anau Mesui", "'Yinka Oyerinde"], 150: ["'Yinka Oyerinde"], 533: ["A'fza Shafie", "Alfredo D'Elia"], 270: ["A'lishia Bowman"], 402: ["A'na Wang"], 167: ["A'na Wang"], 138: ["A'na Wang"], 116: ["A'zraa Afhzan Ab Rahim"], 555: ["A. A. Pukhal'skii"], 347: ["A. Al-Ta'ani", "Abdallah M'hamed"], 241: ["A. B. Ugol'nikov"], 357: ["A. D'Agapayeff", "Alessandro Dell'Orto", "Artur S. d'Avila Garcez", "Bara' Nazzal", "Bernard N'Kaoua"], 553: ["A. D'Alascio"], 449: ["A. D'Alessandro", "Arnold James D'Souza", "Arthur O'neill"], 658: ["A. D'Amico"], 467: ["A. D'Amico", "Antonino D'Alessandro", "Arnaldo D'Amico"], 502: ["A. D'Arminio Monforte", "Artur S. d'Avila Garcez", "Aurèle N'dja"], 407: ["A. D'Orta"], 208: ["A. Dell'Accio", "Andrea Dall'Alba"], 181: ["A. Dell'Acqua"], 182: ["A. Dell'Aglio", "Amit Re'em", "Anas Alaoui M'Darhri", "Andrea D'Andrea", "Andrea Dall'Asta"], 231: ["A. Dell'Arte"], 432: ["A. G. O'Neill", "Alan O'Riordan", "Alessandro D'Amato", "Anderson C. Sant'Ana", "Angela D'An

In [13]:
dict_to_list_of_list = []
for i in range(1,len(unique_conference)+1):
    dict_to_list_of_list.append(final_transformation[i])
print(dict_to_list_of_list)

[["A. M'zah", "Bert Huis in 't Veld"], ["Andrea D'Eusanio"], ["Ahuva Mu'alem", "Alessandro D'Ausilio", "Anthony B. O'Hare", "Artur S. d'Avila Garcez"], ["Anthony M. D'Amato", "Bart D'Hooghe"], ["Artur S. d'Avila Garcez"], ["Artur S. d'Avila Garcez"], ["Anita Pinheiro Sant'Anna"], ["Alessandro D'Amelio"], ["Alexander O'Connor"], ["Arra'di Nur Rizal"], ["Ashwin Geet D'Sa"], ["Alexander O'Connor"], ["Alison O'Neil"], ["Benjamin Ka-Yin T'sou"], ["Ashwin Geet D'Sa"], ["Bara' Al-Jawarneh"], ["Afonso O'Neill"], ["Bernice d'Anjou"], ["Bill O'Connell"], ["Allison D'Eugenio"], ["A. O'Brien"], ["Alejandro D'Ermiliis"], ["Aisling O'Driscoll"], ["Alessandro D'Innocenzo"], ["Bedr'Eddine Ainseba"], ["B. T. O'Donald", "B. V. O'Brien"], ["Artur S. d'Avila Garcez"], ["Alexander O'Connor"], ["Alan O'Donnell"], ["Adrian d'Alessandro", "Ala'a Alslaity"], ["Bara'a Ali Attea"], ["Andrea D'Ambrogio"], ["Alex 'Sandy' Pentland"], ["Alicia D'Anjou", "Annalisa D'Andrea", "Annarita D'Addabbo", "Antonio D'Angelo", 

In [14]:
# Convert dataset to DataFrame
te = TransactionEncoder()
te_ary = te.fit(dict_to_list_of_list).transform(dict_to_list_of_list)
df_final = pd.DataFrame(te_ary, columns=te.columns_)

In [19]:
fpgrowth(df_final, min_support=0.005, use_colnames=True)

,support,itemsets
0,0.005571,(Andrea D'Eusanio)
1,0.045961,(Artur S. d'Avila Garcez)
2,0.015320,(Ahuva Mu'alem)
3,0.005571,(Alessandro D'Ausilio)
4,0.005571,(Anthony B. O'Hare)
...,...,...
68,0.005571,(Abdallah M'hamed)
69,0.005571,(Ala'a Al-Momani)
70,0.005571,"(Alicia D'Anjou, Artur S. d'Avila Garcez)"
71,0.006964,"(Alicia D'Anjou, Annarita D'Addabbo)"
